In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Scipy not supported!


**Only need run the below line on a fresh machine install**

In [ ]:
# this only needs to be run once on a machine if the following error is encountered:
# Resource u'tokenizers/punkt/english.pickle' not found.  Please use the NLTK Downloader to obtain the resource
nltk.download('punkt')

In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

29 documents
10 classes ['casinotab', 'changesubscription', 'circledgame', 'goodbye', 'greeting', 'missingemails', 'playercontact', 'playerpaying', 'subscriptionmonth', 'thanks']
63 unique stemmed words ["'s", '.', 'a', 'anyon', 'ar', 'as', 'been', 'bye', 'can', 'casino', 'chang', 'circ', 'com', 'day', 'did', 'do', 'downgrad', 'email', 'explain', 'for', 'ful', "g'day", 'gam', 'get', 'good', 'goodby', 'has', 'hello', 'help', 'hi', 'how', 'howd', 'i', 'is', 'lat', 'mark', 'me', 'miss', 'mon', 'my', "n't", 'on', 'paid', 'pay', 'play', 'receiv', 'renew', 'respond', 'see', 'subscrib', 'tab', 'thank', 'that', 'the', 'ther', 'to', 'turn', 'uncirc', 'up', 'upgrad', 'what', 'why', 'you']


In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.49812 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.49812 - acc: 0.9408 -- iter: 24/29
Training Step: 4000  | total loss: 0.45010 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.45010 - acc: 0.9467 -- iter: 29/29
--
INFO:tensorflow:C:\Users\Gavin\Documents\jupyter\TensorFlow Chat Bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("i want to change my subscription", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
['casinotab', 'changesubscription', 'circledgame', 'goodbye', 'greeting', 'missingemails', 'playercontact', 'playerpaying', 'subscriptionmonth', 'thanks']


In [8]:
print(model.predict([p]))

[[  2.65320647e-03   6.87322676e-01   2.03397847e-03   2.40192747e-07
    2.97334403e-01   1.62879569e-05   4.63965279e-07   1.02922646e-02
    3.42132204e-04   4.26174256e-06]]


In [9]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )